In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from datetime import datetime
from census import Census
from us import states


## Extract Superfund Sites - NPL data into a DataFrame

In [6]:
sf = pd.read_csv("resources/superfundsites.csv" , dtype={'Zip': str}) #, "Latitude":float
sf["Latitude"] = sf["Latitude"].str.decode("unicode_escape")
sf["Longitude"] = sf["Longitude"].str.decode("unicode_escape")
sf["Latitude"] = sf["Latitude"].str.encode("ascii", "ignore")
sf["Longitude"] = sf["Longitude"].str.encode("ascii", "ignore")
# this showed that these had empty values
# sf["Latitude"].sort_values(ascending=True).head()s
sf=sf.drop(sf.index[1104]) #these did not have lat and long
sf=sf.drop(sf.index[756])
# saw NaNs by scrolling to bottom of this list
# sf["Longitude"].sort_values()
sf=sf.drop(sf.index[1335]) #these were NaN
sf=sf.drop(sf.index[1334])
sf["Converted Date"] = ""
for index, row in sf.iterrows():
    sf.set_value(index, "Converted Date", datetime.strptime(row["Final Date"], '%m/%d/%y'))
sf["Year"]=""
for index, row in sf.iterrows():
    sf.set_value(index, "Year", row["Converted Date"].year)
sf.head()

,Reg,St,Site Name,Unnamed: 3,Site ID,EPA ID,Address,City,Zip,County,Unnamed: 10,Unnamed: 11,FF?,Latitude,Longitude,Final Date,Unnamed: 16,Unnamed: 17,Converted Date,Year
0,‭01‬,CT,BARKHAMSTED-NEW HARTFORD LANDFILL,NaN,‭0100255‬,CTD980732333,ROUTE 44,BARKHAMSTED,06063,LITCHFIELD,NaN,NaN,N,b'41.980550',b'-073.047220 ',10/04/89,NaN,NaN,1989-10-04 00:00:00,1989
1,‭01‬,CT,BEACON HEIGHTS LANDFILL,NaN,‭0100180‬,CTD072122062,BLACKBERRY HILL ROAD,BEACON FALLS,06403,NEW HAVEN,NaN,NaN,N,b'41.431950',b'-073.035280 ',09/08/83,NaN,NaN,1983-09-08 00:00:00,1983
2,‭01‬,CT,DURHAM MEADOWS,NaN,‭0100108‬,CTD001452093,124 MAIN ST,DURHAM,06422,MIDDLESEX,NaN,NaN,N,b'41.481110',b'-072.681380 ',10/04/89,NaN,NaN,1989-10-04 00:00:00,1989
3,‭01‬,CT,GALLUP'S QUARRY,NaN,‭0100201‬,CTD108960972,ROUTE 12,PLAINFIELD,06374,WINDHAM,NaN,NaN,N,b'41.665280',b'-071.924160 ',10/04/89,NaN,NaN,1989-10-04 00:00:00,1989
4,‭01‬,CT,KELLOGG-DEERING WELL FIELD,NaN,‭0100252‬,CTD980670814,NORWALK WATER DEPARTMENT,NORWALK,06856,FAIRFIELD,NaN,NaN,N,b'41.130550',b'-073.431950 ',09/21/84,NaN,NaN,1984-09-21 00:00:00,1984


## Extract Census data for 2015 for all US zip codes and store in a DataFrame

In [4]:
# Need to run for other years (if needed)
c = Census("85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98", year=2015)

census_data = c.acs5.get(("NAME", "B19013_001E", "B19301_001E", "B23025_002E", "B23025_007E", "B23025_003E", "B23025_004E", "B23025_005E", "B23025_006E", "C24010_003E", "C24010_004E", "C24010_005E", "C24010_006E", "C24010_007E", "C24010_008E", "C24010_009E", "C24010_010E", "C24010_011E", "C24010_012E", "C24010_013E", "C24010_014E", "C24010_015E", "C24010_016E", "C24010_017E", "C24010_018E", "C24010_019E", "C24010_020E", "C24010_021E", "C24010_022E", "C24010_023E", "C24010_024E", "C24010_025E", "C24010_026E", "C24010_027E", "C24010_028E", "C24010_029E", "C24010_030E", "C24010_031E", "C24010_032E", "C24010_033E", "C24010_034E", "C24010_035E", "C24010_036E", "C24010_037E", "C24010_039E", "C24010_040E", "C24010_041E", "C24010_042E", "C24010_043E", "C24010_044E", "C24010_045E", "C24010_046E", "C24010_047E", "C24010_048E", "C24010_049E", "C24010_050E", "C24010_051E", "C24010_052E", "C24010_053E", "C24010_054E", "C24010_055E", "C24010_056E", "C24010_057E", "C24010_058E", "C24010_059E", "C24010_060E", "C24010_061E", "C24010_062E", "C24010_063E", "C24010_064E", "C24010_065E", "C24010_066E", "C24010_067E", "C24010_068E", "C24010_069E", "C24010_070E", "C24010_071E", "C24010_072E", "C24010_073E", "B17001_002E", "B17001_003E", "B17001_017E", "B17001A_002E", "B17001B_002E", "B17001C_002E", "B17001D_002E", "B17001E_002E", "B17001G_002E", "B17001I_002E", "B17012_002E", "B17012_003E", "B17012_009E", "B17012_014E", "B01002_001E", "B01002_002E", "B01002_003E", "B01003_001E", "B02001_002E", "B02001_003E", "B02001_004E", "B02001_005E", "B02001_006E", "B02001_008E", "B03001_003E", "B25035_001E", "B25058_001E", "B25064_001E", "B25077_001E", "B25088_002E", "B08136_003E", "B08136_004E", "B08136_007E", "B08136_011E", "B08136_012E", "B08301_001E", "B08301_003E", "B08301_004E", "B08301_010E", "B08301_019E", "B08101_041E", "B15003_002E", "B15003_017E", "B15003_018E", "B15003_021E", "B15003_022E", "B15003_023E", "B15003_024E", "B15003_025E", "B16001_002E", "B16001_003E", "B16001_006E", "B16001_009E", "B16001_012E", "B16001_015E", "B16001_018E", "B16001_021E", "B16001_030E", "B16001_033E", "B16001_036E", "B16001_039E", "B16001_045E", "B16001_048E", "B16001_051E", "B16001_054E", "B16001_057E", "B16001_066E", "B16001_069E", "B16001_072E", "B16001_075E", "B16001_078E", "B16001_081E", "B16001_084E", "B16001_087E", "B16001_093E", "B16001_099E", "B16001_105E", "B16001_108E", "B16001_111E"), {'for': 'zip code tabulation area:*'})
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)
# Save as a csv (if needed)
# Note to avoid any issues later, use encoding="utf-8"
# census_pd.to_csv("zipCode_census_data.csv", encoding="utf-8", index=False)
census_pd.head()

,B01002_001E,B01002_002E,B01002_003E,B01003_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_008E,...,C24010_066E,C24010_067E,C24010_068E,C24010_069E,C24010_070E,C24010_071E,C24010_072E,C24010_073E,NAME,zip code tabulation area
0,37.6,36.5,39,17982,16956,102,14,0,0,64,...,14,0,0,14,125,110,0,15,ZCTA5 00601,00601
1,39.5,38.6,40.3,40260,23144,693,8,72,0,14798,...,15,0,0,15,376,334,0,42,ZCTA5 00602,00602
2,39.9,37.7,41.4,52408,36177,2057,76,655,0,7801,...,43,0,11,32,211,128,0,83,ZCTA5 00603,00603
3,40.8,41,40.3,6331,4399,151,0,0,0,126,...,15,0,0,15,93,93,0,0,ZCTA5 00606,00606
4,40.2,36.9,42.2,28328,17791,950,0,26,0,3355,...,0,0,0,0,788,738,15,35,ZCTA5 00610,00610
